In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

%matplotlib inline

# Algorithm Chains and Pipelines

In [ ]:
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# load and split the data
cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(
    cancer.data, cancer.target, random_state=0)

# compute minimum and maximum on the training data
scaler = MinMaxScaler().fit(X_train)
# rescale training data
X_train_scaled = scaler.transform(X_train)

svm = SVC()
# learn an SVM on the scaled training data
svm.fit(X_train_scaled, y_train)
# scale test data and score the scaled data
X_test_scaled = scaler.transform(X_test)
svm.score(X_test_scaled, y_test)

### Building Pipelines

In [ ]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([("scaler", MinMaxScaler()), ("svm", SVC())])

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
pipe.score(X_test, y_test)

### Using Pipelines in Grid-searches

In [ ]:
param_grid = {'svm__C': [0.001, 0.01, 0.1, 1, 10, 100],
              'svm__gamma': [0.001, 0.01, 0.1, 1, 10, 100]}

In [ ]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, param_grid=param_grid, cv=5)
grid.fit(X_train, y_train)
print("best cross-validation accuracy:", grid.best_score_)
print("test set score: ", grid.score(X_test, y_test))
print("best parameters: ", grid.best_params_)

# Not using Pipelines vs feature selection

In [ ]:
rnd = np.random.RandomState(seed=0)
X = rnd.normal(size=(100, 10000))
y = rnd.normal(size=(100,))

In [ ]:
from sklearn.feature_selection import SelectPercentile, f_regression

select = SelectPercentile(score_func=f_regression, percentile=5).fit(X, y)
X_selected = select.transform(X)
print(X_selected.shape)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
np.mean(cross_val_score(Ridge(), X_selected, y, cv=5))

In [ ]:
pipe = Pipeline([("select", SelectPercentile(score_func=f_regression, percentile=5)),
                 ("ridge", Ridge())])
np.mean(cross_val_score(pipe, X, y, cv=5))

### The General Pipeline Interface

In [ ]:
def fit(self, X, y):
    X_transformed = X
    for step in self.steps[:-1]:
        # iterate over all but the final step
        # fit and transform the data
        X_transformed = step[1].fit_transform(X_transformed, y)
    # fit the last step
    self.steps[-1][1].fit(X_transformed, y)
    return self

In [ ]:
def predict(self, X):
    X_transformed = X
    for step in self.steps[:-1]:
        # iterate over all but the final step
        # transform the data
        X_transformed = step[1].transform(X_transformed)
    # predict using the last step
    return self.steps[-1][1].predict(X_transformed)

### Convenient Pipeline creation with ``make_pipeline``

In [ ]:
from sklearn.pipeline import make_pipeline
# standard syntax
pipe_long = Pipeline([("scaler", MinMaxScaler()), ("svm", SVC(C=100))])
# abbreviated syntax
pipe_short = make_pipeline(MinMaxScaler(), SVC(C=100))

In [ ]:
pipe_short.steps

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

pipe = make_pipeline(StandardScaler(), PCA(n_components=2), StandardScaler())
pipe.steps

#### Accessing step attributes

In [ ]:
# fit the pipeline defined above to the cancer dataset
pipe.fit(cancer.data)
# extract the first two principal components from the "pca" step
components = pipe.named_steps.pca.components_
print(components.shape)

In [ ]:
pipe.named_steps['standardscaler-1']

#### Accessing attributes in grid-searched pipeline.

In [ ]:
from sklearn.linear_model import LogisticRegression

pipe = make_pipeline(StandardScaler(), LogisticRegression())

In [ ]:
param_grid = {'logisticregression__C': [0.01, 0.1, 1, 10, 100]}

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    cancer.data, cancer.target, random_state=4)
grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(X_train, y_train)

In [ ]:
grid.predict(X_test)

In [ ]:
print(grid.best_estimator_)

In [ ]:
print(grid.best_estimator_.named_steps.logisticregression)

In [ ]:
print(grid.best_estimator_.named_steps.logisticregression.coef_)

### Grid-searching preprocessing steps and model parameters

In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()
X_train, X_test, y_train, y_test = train_test_split(
    boston.data, boston.target, random_state=0)

from sklearn.preprocessing import PolynomialFeatures
pipe = make_pipeline(
    StandardScaler(),
    PolynomialFeatures(),
    Ridge())

In [ ]:
param_grid = {'polynomialfeatures__degree': [1, 2, 3],
              'ridge__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

In [ ]:
grid = GridSearchCV(pipe, param_grid=param_grid,
                    cv=5, n_jobs=-1, return_train_score=True)
grid.fit(X_train, y_train)

In [ ]:
res = pd.DataFrame(grid.cv_results_)
res.head()

In [ ]:
res = pd.pivot_table(res, index=['param_polynomialfeatures__degree', 'param_ridge__alpha'],
               values=['mean_train_score', 'mean_test_score'])

In [ ]:
res['mean_test_score'].unstack()

In [ ]:
res['mean_train_score'].unstack()

In [ ]:
print(grid.best_params_)

In [ ]:
grid.score(X_test, y_test)

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import RepeatedKFold

In [ ]:
pipe = Pipeline([('scaler', StandardScaler()), ('regressor', Ridge())])
param_grid = {'scaler': [StandardScaler(), MinMaxScaler(), None],
              'regressor': [Ridge(), Lasso()],
              'regressor__alpha': np.logspace(-3, 3, 7)}
grid = GridSearchCV(pipe, param_grid, cv=RepeatedKFold(n_splits=10, n_repeats=10))
grid.fit(X_train, y_train)
grid.score(X_test, y_test)

In [ ]:
grid.best_score_

In [ ]:
grid.best_params_

In [ ]:
from sklearn.tree import DecisionTreeRegressor
param_grid = [{'regressor': [DecisionTreeRegressor()], 'regressor__max_depth': [2, 3, 4]},
              {'regressor': [Ridge()], 'regressor__alpha': [0.1, 1]}
             ]

# More on ColumnTransformer

In [ ]:
from sklearn.compose import make_column_transformer, ColumnTransformer
bike = pd.read_csv("data/bike_day_raw.csv")
bike.head()

In [ ]:
bike.dtypes

In [ ]:
bike_data = bike.drop("cnt", axis=1)
cat_features = bike.columns[:6]
cat_features

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ct = make_column_transformer((OneHotEncoder(sparse=False), cat_features),
                             remainder=StandardScaler())
ct.transformers

In [ ]:
ct.fit(bike_data)
ct.transformers_

In [ ]:
ct.transform(bike_data).shape

In [ ]:
ColumnTransformer([('ohe', OneHotEncoder(sparse=False), cat_features)],
                  remainder=StandardScaler())

ColumnTransformer([('ohe', OneHotEncoder(sparse=False), cat_features),
                   ('scaler', StandardScaler(), [6, 7, 8, 9])])

In [ ]:
ct = make_column_transformer((OneHotEncoder(sparse=False), cat_features),
                             remainder=StandardScaler())
ohe_pipe = make_pipeline(ct, Ridge())

In [ ]:
bike['windspeed'].hist()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(bike_data, bike.cnt, random_state=42)

In [ ]:
from sklearn.preprocessing import PowerTransformer

ct = make_column_transformer((OneHotEncoder(sparse=False), cat_features))
ohe_pipe = make_pipeline(ct, Ridge())

param_grid = {'columntransformer__remainder':
              [StandardScaler(), PowerTransformer(method='yeo-johnson')],
              'ridge__alpha': np.logspace(-3, 2, 6)}
grid = GridSearchCV(ohe_pipe, param_grid, cv=5)
grid.fit(X_train, y_train)
grid.score(X_test, y_test)

In [ ]:
grid.best_params_

In [ ]:
res = pd.DataFrame(grid.cv_results_)
res

In [ ]:
plt.plot(res.mean_test_score[:6].values, label="StandardScaler")
plt.plot(res.mean_test_score[6:].values, label="PowerTransformer")
plt.legend()

# Exercise

Load the adult dataset. Create a pipline using the ColumnTransformer, OneHotEncoder, Scaling, and polynomial features and a linear classifier.

Search over the best options for the polynomial features together with the regularization of a linear model.